In [1]:
# Test if GSM model can make one or more metabolites
import pandas as pd
import cobra
from collections import OrderedDict
from copy import deepcopy
import os

#### Load data

In [7]:
curr_dir = os.getcwd()
# Metabolic model (COBRApy json)
OGmodel = cobra.io.load_json_model(curr_dir + '/../iRhtoCN.json')
biomass = 'BIOMASS'

In [32]:
# List of rxns to check
kolist = set(['CHTNDA_c','SDPDS_c','TKT1_c','TKT2_c','XU5PFGT_x'])
Fsolutions = []
IFsolutions = []
# print("demands", model.demands)

In [33]:
OGmodel.medium

{'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 5.0,
 'EX_fe2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_btn_c': 0.01}

# Check each metabolite and show all rxns involved in its production

In [34]:
import copy
with model: #to prevent unintended edits
    # Create a copy of the model to avoid modifying the original model
    model = copy.deepcopy(OGmodel)
    # Get the reaction object by ID
    # Knock out the reaction by setting its bounds to zero

    # Perform FBA on the modified model
    model.objective_direction = "max"
    model.objective = model.reactions.get_by_id(biomass)

    for rxn in kolist:
        # model.reactions.rxn.knock_out()
        print("Objective: ",model.objective)
        reaction = model.reactions.get_by_id(rxn)
        reaction.lower_bound = 0.0
        reaction.upper_bound = 0.0
        print("knocked out ",rxn)
        fba = model.optimize()
        #fba = cobra.flux_analysis.optimize_minimal_flux(model)
        print(fba)
        if fba.objective_value == 0:
            # in case rxn flux can only be non-positive, minimize to check for nonzero fluxes
            model.objective_direction = "min"
            fba = model.optimize()
            print(fba)
            if fba.objective_value == 0:
                # add to list of unusable metabolites
                IFsolutions.append(rxn)
            else:
                # note rxn and objective value
                Fsolutions.append(rxn)
                print(str(fba.fluxes[rxn]))
        else: 
            Fsolutions.append(rxn)
            print(str(fba.fluxes[rxn]))
    fba = model.optimize()
    print(fba)
    if fba.objective_value == 0:
        print("Growth NOT possible without all knocked out rxns")
    else: 
        print("Growth possible without all knocked out rxns")
    print("Growth possible without ",Fsolutions)
    print("Growth NOT possible without ",IFsolutions)

Objective:  Maximize
1.0*BIOMASS_RT_CLIM - 1.0*BIOMASS_RT_CLIM_reverse_12b42
knocked out  TKT2_c
<Solution 0.048 at 0x1b6c9b790>
0.0
Objective:  Maximize
1.0*BIOMASS_RT_CLIM - 1.0*BIOMASS_RT_CLIM_reverse_12b42
knocked out  CHTNDA_c
<Solution 0.048 at 0x18ddc2e60>
0.0
Objective:  Maximize
1.0*BIOMASS_RT_CLIM - 1.0*BIOMASS_RT_CLIM_reverse_12b42
knocked out  SDPDS_c
<Solution 0.048 at 0x1b6c98190>
0.0
Objective:  Maximize
1.0*BIOMASS_RT_CLIM - 1.0*BIOMASS_RT_CLIM_reverse_12b42
knocked out  TKT1_c
<Solution 0.000 at 0x19cd04040>
<Solution 0.000 at 0x1b6c98250>
Objective:  Minimize
1.0*BIOMASS_RT_CLIM - 1.0*BIOMASS_RT_CLIM_reverse_12b42
knocked out  XU5PFGT_x
<Solution 0.000 at 0x19fa55420>
<Solution 0.000 at 0x1b6c98460>
<Solution 0.000 at 0x19cd04130>
Growth NOT possible without all knocked out rxns
Growth possible without  ['TKT2_c', 'CHTNDA_c', 'SDPDS_c']
Growth NOT possible without  ['TKT1_c', 'XU5PFGT_x']


In [38]:
# Test if GSM model can make one or more metabolites
import pandas as pd
import cobra
from collections import OrderedDict
import copy
from copy import deepcopy
import os
from itertools import combinations

curr_dir = os.getcwd()
# Metabolic model (COBRApy json)
model = cobra.io.load_json_model(curr_dir + '/../iRhtoCN.json')
biomass = 'BIOMASS_RT_CLIM'
# Define the list of reactions to knock out
# all_reactions = ["reaction1_id", "reaction2_id", "reaction3_id"]
all_reactions = set(['CHTNDA_c','RPE_c','SDPDS_c','TKT1_c','TKT2_c','XU5PFGT_x'])

# Function to perform FBA and categorize reactions
def categorize_reactions(model, reactions_to_knockout, essential_reactions, feasible_knockouts):
    # Create a copy of the model to avoid modifying the original model
    model_copy = model.copy()

    # Knock out the reactions in the current combination
    for reaction_id in reactions_to_knockout:
        reaction = model_copy.reactions.get_by_id(reaction_id)
        reaction.lower_bound = 0.0
        reaction.upper_bound = 0.0

    # Perform FBA on the modified model
    model_copy.objective = model.reactions.get_by_id(biomass)
    model_copy.objective_direction = "max"
    knockout_solution = model_copy.optimize()
    # knockout_solution = cobra.flux_analysis.optimize_minimal_flux(model_copy)

    # Check the objective value
    if knockout_solution.objective_value <= 0:
        essential_reactions.extend(reactions_to_knockout)
    else:
        feasible_knockouts.append(reactions_to_knockout)

# Lists to store results
essential_reactions = []
feasible_single_knockouts = []
feasible_knockout_combinations = [feasible_single_knockouts]

# Try single knockouts
for reaction_id in all_reactions:
    categorize_reactions(model, [reaction_id], essential_reactions, feasible_single_knockouts)

# Function to generate higher-order knockout combinations
def generate_higher_order_combinations(feasible_knockouts, n):
    higher_order_combinations = []
    for combo in combinations(feasible_knockouts, n):
        combined_knockout = [item for sublist in combo for item in sublist]
        print(combined_knockout) # for testing
        higher_order_combinations.append(combined_knockout)
    return higher_order_combinations

# feasible_double_knockouts = []
# # Try double knockouts
# for combo in combinations(all_reactions, 2):
#     combined_knockout = list(combo)
#     categorize_reactions(model, combined_knockout, essential_reactions, feasible_double_knockouts)

# Try higher-order knockouts (e.g., triple, quadruple)
for order in range(2, len(all_reactions)):
    higher_order_combinations = generate_higher_order_combinations(feasible_knockout_combinations[-1], order)
    feasible_knockout_combinations.append([])
    for combo in higher_order_combinations:
        categorize_reactions(model, combo, essential_reactions, feasible_knockout_combinations[-1])

# Print the results
print("Essential Reactions:")
print(essential_reactions)
print("\nFeasible Single Knockouts:")
print(feasible_single_knockouts)
print("\nFeasible Double Knockouts:")
print(feasible_double_knockouts)
print("\nFeasible Knockout Combinations:")
for i, combinations in enumerate(feasible_knockout_combinations):
    print(f"Order-{i + 2} Knockouts:")
    print(combinations)


['TKT2_c', 'CHTNDA_c']
['TKT2_c', 'RPE_c']
['TKT2_c', 'SDPDS_c']
['TKT2_c', 'TKT1_c']
['TKT2_c', 'XU5PFGT_x']
['CHTNDA_c', 'RPE_c']
['CHTNDA_c', 'SDPDS_c']
['CHTNDA_c', 'TKT1_c']
['CHTNDA_c', 'XU5PFGT_x']
['RPE_c', 'SDPDS_c']
['RPE_c', 'TKT1_c']
['RPE_c', 'XU5PFGT_x']
['SDPDS_c', 'TKT1_c']
['SDPDS_c', 'XU5PFGT_x']
['TKT1_c', 'XU5PFGT_x']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'TKT2_c', 'XU5PFGT_x']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'CHTNDA_c', 'RPE_c']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'CHTNDA_c', 'SDPDS_c']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'CHTNDA_c', 'TKT1_c']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'CHTNDA_c', 'XU5PFGT_x']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'RPE_c', 'SDPDS_c']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'RPE_c', 'TKT1_c']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'RPE_c', 'XU5PFGT_x']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'SDPDS_c', 'TKT1_c']
['TKT2_c', 'CHTNDA_c', 'TKT2_c', 'SDPDS_c', 'SDPDS_c', 'XU5PFGT_x']


Exception ignored in: <function Cplex.__del__ at 0x10e3aa200>
Traceback (most recent call last):
  File "/Users/ejm6426/anaconda3/envs/work1a/lib/python3.10/site-packages/cplex/__init__.py", line 923, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 